# COVID-19 Modeling

### Step 1: Create dataset with a new column for "id" (which we will fill in the next step!)

In [1]:
!bq --location=US mk --dataset covid_19_model

Dataset 'arcane-footing-266618:covid_19_model' successfully created.


In [2]:
%%bigquery
create or replace table covid_19_model.Cases
as select null as id, * from covid_19_staging.Cases

""


### Step 2: Fill id column using hash function FARM_FINGERPRINT to compute a unique hash for the location string so that our table can have a Primary Key

In [4]:
%%bigquery
update covid_19_model.Cases set id = FARM_FINGERPRINT(country) 
where state is null

""


In [5]:
%%bigquery
update covid_19_model.Cases set id = FARM_FINGERPRINT(concat(state, country)) 
where state is not null

""


Make sure id field has no null values!

In [6]:
%%bigquery
select count(*) as count_null_ids
from covid_19_model.Cases
where id is null

,count_null_ids
0,0


And let's check out what our id field looks like:

In [7]:
%%bigquery
select id, state, country
from covid_19_model.Cases
order by state, country
limit 5

,id,state,country
0,8576431891811451300,None,Azerbaijan
1,8778414404485170876,None,Afghanistan
2,8778414404485170876,None,Afghanistan
3,8778414404485170876,None,Afghanistan
4,8778414404485170876,None,Afghanistan


### Step 3: Split table because there is redundant information (region, state, lat/long)

We create a table for Location because that information is replicated every day (we will manipulate this table later.)

In [8]:
%%bigquery
create or replace table covid_19_model.Location_SQL_1
as select distinct id, state, country, latitude, longitude, fips, admin2, combined_key
from covid_19_model.Cases

""


Let's create an Event table as well. We can associate Events with Locations via the id.

In [9]:
%%bigquery
create or replace table covid_19_model.Event_SQL_1
as select id as location_id, last_update, confirmed, deaths, recovered, active
from covid_19_model.Cases

""


Below, we can see that there is a change in the way the `last_update` field is represented from old records to new records (goes from YYYY to YY).

In [12]:
%%bigquery
select * from covid_19_model.Event_SQL_1
order by last_update
limit 5

,location_id,last_update,confirmed,deaths,recovered,active
0,3093811823925351433,1/22/2020 17:00,1.0,None,None,None
1,2544652828731166483,1/22/2020 17:00,1.0,None,None,None
2,400699263222839825,1/22/2020 17:00,NaN,None,None,None
3,-8459520092734636284,1/22/2020 17:00,5.0,None,None,None
4,3061248092517028102,1/22/2020 17:00,2.0,None,None,None


In [13]:
%%bigquery
select * from covid_19_model.Event_SQL_1
order by last_update desc
limit 5

,location_id,last_update,confirmed,deaths,recovered,active
0,6443493987885756991,4/6/20 9:37,914,4,216,694
1,9155895331965305746,4/6/20 6:20,373,5,57,311
2,-3396123447326000985,4/6/20 5:30,536,6,389,141
3,-6225137598979003815,4/6/20 2:36,139,2,132,5
4,-4927258461359090633,4/6/20 2:21,67803,3212,64014,577


### Step 4: Standardize the timestamps in the Event table using SQL

Let's compare the records that have '/' instead of '-' format for the dates.

In [14]:
%%bigquery
create or replace table covid_19_model.Event_SQL_2 as
select *
from covid_19_model.Event_SQL_1
where strpos(last_update, '/') > 0

""


In [15]:
%%bigquery
create or replace table covid_19_model.Event_SQL_3 as
select location_id, cast(last_update as datetime) last_update, confirmed, deaths, recovered, active 
from covid_19_model.Event_SQL_1
where strpos(last_update, '-') > 0

""


In [16]:
%%bigquery
select count(*) as count_timestamp_slash
from covid_19_model.Event_SQL_2

,count_timestamp_slash
0,12054


In [17]:
%%bigquery
select count(*) as count_timestamp_dash
from covid_19_model.Event_SQL_3

,count_timestamp_dash
0,90609


Above, we can see that there are many more records that use '-' rather than '/', so let's standardize to a dash. We can start by getting the length of the string for each year.

In [1]:
%%bigquery
select last_update, length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) as year_length
from
(select distinct last_update
from covid_19_model.Event_SQL_2)

,last_update,year_length
0,3/8/20 5:19,2
1,3/8/20 5:31,2
2,4/1/20 1:36,2
3,4/2/20 0:09,2
4,4/2/20 0:16,2
...,...,...
126,1/22/2020 17:00,4
127,1/31/2020 10:37,4
128,1/31/2020 15:20,4
129,1/31/2020 16:13,4


Now we can standardize the timestamps using the length we found above for each year and `parse_datetime()` which uses a format string and the date string to return a datetime in the format YYYY-MM-DD HH:MM:SS.

In [2]:
%%bigquery
select last_update, parse_datetime('%m/%d/%y %H:%M', last_update) as last_update_datetime
from
(select distinct last_update
from covid_19_model.Event_SQL_2)
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2

,last_update,last_update_datetime
0,3/8/20 5:19,2020-03-08 05:19:00
1,3/8/20 5:31,2020-03-08 05:31:00
2,4/1/20 1:36,2020-04-01 01:36:00
3,4/2/20 0:09,2020-04-02 00:09:00
4,4/2/20 0:16,2020-04-02 00:16:00
...,...,...
95,3/30/20 12:30,2020-03-30 12:30:00
96,3/30/20 22:52,2020-03-30 22:52:00
97,3/30/20 22:58,2020-03-30 22:58:00
98,3/31/20 13:54,2020-03-31 13:54:00


We can use what we just created for year strings that had lengths of 2 and 4. Let's create a new table called Event_SQL_4 that brings all of this data together in a clean way.

In [3]:
%%bigquery
create or replace table covid_19_model.Event_SQL_4 as
select location_id, parse_datetime('%m/%d/%y %H:%M', last_update) as last_update, confirmed, deaths, recovered, active
from covid_19_model.Event_SQL_2
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2
union all
select location_id, parse_datetime('%m/%d/%Y %H:%M', last_update) as last_update, confirmed, deaths, recovered, active
from covid_19_model.Event_SQL_2
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 4

""


In [5]:
%%bigquery
select distinct last_update 
from covid_19_model.Event_SQL_4
order by last_update

,last_update
0,2020-01-22 17:00:00
1,2020-01-23 17:00:00
2,2020-01-24 17:00:00
3,2020-01-25 17:00:00
4,2020-01-26 16:00:00
...,...
126,2020-04-06 06:20:00
127,2020-04-06 09:37:00
128,2020-04-06 23:21:00
129,2020-04-06 23:22:00


Now we can bring our original '-' table with our newly created table and union them into a single table, Event_SQL_5. We can check the number of records we have to make sure it adds up to the sum from the two original tables.

In [6]:
%%bigquery
create or replace table covid_19_model.Event_SQL_5 as
select * from covid_19_model.Event_SQL_3
union all
select * from covid_19_model.Event_SQL_4

""


In [8]:
%%bigquery
select count(*) total_event_count
from covid_19_model.Event_SQL_5

,total_event_count
0,102663


We should remove all the duplicate records in our table. You can see that around 35,000 records get eliminated from this!

In [9]:
%%bigquery
create or replace table covid_19_model.Event_SQL_5 as
select distinct location_id, last_update, confirmed, deaths, recovered, active from covid_19_model.Event_SQL_5

""


In [10]:
%%bigquery
select count(*) as total_event_count from covid_19_model.Event_SQL_5

,total_event_count
0,68411


Now, we have some records that are different BUT they have the same location and timestamp.

In [12]:
%%bigquery
select location_id, last_update, count(*) as duplicate_events
from covid_19_model.Event_SQL_5
group by location_id, last_update
having count(*) > 1
order by count(*) desc

,location_id,last_update,duplicate_events
0,-5615092215703141091,2020-05-01 02:32:28,138
1,-5615092215703141091,2020-04-30 02:32:27,135
2,-5615092215703141091,2020-04-26 02:30:51,133
3,-5615092215703141091,2020-04-29 02:32:29,132
4,-5615092215703141091,2020-04-24 03:30:50,129
...,...,...,...
1629,4344896043720875999,2020-03-13 12:33:03,2
1630,-7404292298259911451,2020-03-14 16:53:03,2
1631,-9159398279919354893,2020-03-14 22:13:19,2
1632,6479081419518102260,2020-03-14 18:53:03,2


Let's take a look at one of these duplicate records in depth!

In [13]:
%%bigquery
select * from covid_19_model.Event_SQL_5
where location_id = -118082929383066870 and last_update = '2020-03-14 16:53:03'

,location_id,last_update,confirmed,deaths,recovered,active
0,-118082929383066870,2020-03-14 16:53:03,13,0,0,None
1,-118082929383066870,2020-03-14 16:53:03,7,0,0,None


We can see from the two records above that they are actually 1 record (because they have the same location and timestamp) but that one is slightly more up to date (13 cases confirmed rather than just 7 cases confirmed.) We can use `rank()` to rank all these duplicate records, and make a new table that takes only the records that have the highest confirmed cases (because we know that records will be the most "up-to-date").

In [15]:
%%bigquery
select *, 
rank() over (partition by location_id, last_update order by confirmed desc) as rank
from covid_19_model.Event_SQL_5
where location_id in (-118082929383066870, 5199822387082299175)
limit 15

,location_id,last_update,confirmed,deaths,recovered,active,rank
0,-118082929383066870,2020-03-10 02:33:04,4,0,0,NaN,1
1,-118082929383066870,2020-03-11 10:13:20,5,0,0,NaN,1
2,-118082929383066870,2020-03-11 20:00:00,6,0,0,NaN,1
3,-118082929383066870,2020-03-12 21:39:10,6,0,0,NaN,1
4,-118082929383066870,2020-03-14 16:53:03,13,0,0,NaN,1
5,-118082929383066870,2020-03-14 16:53:03,7,0,0,NaN,2
6,-118082929383066870,2020-03-16 22:33:03,17,0,0,NaN,1
7,-118082929383066870,2020-03-17 23:13:10,26,0,0,NaN,1
8,-118082929383066870,2020-03-19 23:43:04,44,0,0,NaN,1
9,-118082929383066870,2020-03-21 00:13:22,55,0,0,NaN,1


In [16]:
%%bigquery
create or replace table covid_19_model.Event_SQL_Final as
select location_id, last_update, confirmed, deaths, recovered, active
from
(select *, 
rank() over (partition by location_id, last_update order by confirmed desc) as rank
from covid_19_model.Event_SQL_5)
where rank = 1

""


We have now successfully modeled our Event table using just some basic SQL commands! As good data engineers, we will check to make sure our primary key is still valid below. You can see that we still have 2 duplicate records, but that's pretty good for having over 13,000 data points.

In [17]:
%%bigquery
select count(*) total_event_count
from covid_19_model.Event_SQL_Final

,total_event_count
0,14738


In [18]:
%%bigquery
select count(*) distinct_event_count
from
(select distinct location_id, last_update
from covid_19_model.Event_SQL_Final)

,distinct_event_count
0,14737
